In [40]:
import pandas as pd 
import numpy as np 
import scipy as scp 

from bs4 import BeautifulSoup 
import requests as rq 
import re

from selenium import webdriver

import platform

In [62]:
lista_Links_Tabela = []
if(platform.system() == 'Windows'):
    browser = webdriver.Chrome(executable_path="../2017-2-projeto-wildcats/chromer/chromedriver.exe")
else:
    browser = webdriver.Chrome()
url = "https://www.reclameaqui.com.br/ranking/"
browser.get(url)
boxes_rankings = browser.find_elements_by_xpath('//div[@class="box-gray"]')
for box in boxes_rankings:
    lista_Empresas_Ranking =box.find_elements_by_xpath('.//li[@class="ng-scope"]')
    #titulo_box = box.find_element_by_xpath("//h2[@class='ng-binding']")
    titulo=box.find_element_by_tag_name('h2')
    for empresa_no_ranking in lista_Empresas_Ranking:
        empresa = empresa_no_ranking.find_element_by_xpath('.//a[@class="business-name ng-binding ng-scope"]')
        link = empresa.get_attribute("href")
        lista_Links_Tabela.append([titulo.text, link])